In [618]:
import pandas as pd
import gensim
from gensim.corpora.Dictionary import Dictionary as d
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel

In [625]:
data = pd.read_csv("training-data-2019-04-05.csv")
data_text= data_text.drop_duplicates(subset='comments', keep="last").reset_index()
data_text = data[['comments']]
data_text['index'] = data_text.index
documents = data_text

/Users/selenacordona/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [691]:
data.head()

,commentsUrl,comments,date,date_str,linkFlair,score,subreddit,title
0,https://old.reddit.com/r/wallstreetbets/commen...,I think this selloff will bleed into next week...,"datetime.datetime(2019, 3, 22, 23, 10, 1, 823...",'2019-03-22',Gain,14,wallstreetbets,UPDATE it worked out
1,https://old.reddit.com/r/wallstreetbets/commen...,What if the sell off was them selling of...,"datetime.datetime(2019, 3, 22, 23, 10, 1, 823...",'2019-03-22',Gain,14,wallstreetbets,UPDATE it worked out
2,https://old.reddit.com/r/wallstreetbets/commen...,Nope Saw someone buy puts at the ask on futur...,"datetime.datetime(2019, 3, 22, 23, 10, 1, 823...",'2019-03-22',Gain,14,wallstreetbets,UPDATE it worked out
3,https://old.reddit.com/r/wallstreetbets/commen...,This interface is super confusing Just tell u...,"datetime.datetime(2019, 3, 22, 23, 10, 1, 823...",'2019-03-22',Gain,14,wallstreetbets,UPDATE it worked out
4,https://old.reddit.com/r/wallstreetbets/commen...,Yolo Microsoft put Made another 13k on RH,"datetime.datetime(2019, 3, 22, 23, 10, 1, 823...",'2019-03-22',Gain,14,wallstreetbets,UPDATE it worked out


In [626]:
print len(documents)
print documents[:5]

27961
                                            comments  index
0  I think this selloff will bleed into next week...      0
1  What if the sell off was    them    selling of...      1
2  Nope  Saw someone buy puts at the ask on futur...      2
3  This interface is super confusing  Just tell u...      3
4      Yolo Microsoft put  Made another 13k on RH         4


### Data Pre-Processing:  Tokenization

In [627]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2019)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/selenacordona/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# I thought this would make a difference, but it just adds random words to the topics...
data_text.title = \
data_text.title.apply(lambda x: re.sub(r"(\d+)", \
                                          lambda x: num2words.num2words(int(x.group(0))), x))


Create a function to run lemmatization and stemming on a given dataset.

In [631]:
# tokenization + lemmatization step
def lemmatization(token):
    w = WordNetLemmatizer()
    sanitized_token = w.lemmatize(token)
    #SnowballStemmer()
    stemmer = SnowballStemmer("english")
    return stemmer.stem(sanitized_token)

def tokenization(text):
    result=[]
    tokens = gensim.utils.simple_preprocess(text)
    for token in tokens:
        if token not in gensim.parsing.preprocessing.STOPWORDS and \
        len(token) > 3:
            result.append(lemmatization(token))
    return result

In [572]:
data_text = data_text.reset_index()

In [633]:
data_text = data_text[['comments']]

In [634]:
tokenization(data_text.comments[1035])

[u'similar',
 u'general',
 u'posit',
 u'sentiment',
 u'knew',
 u'googl',
 u'game',
 u'stream',
 u'keynot',
 u'come',
 u'talk',
 u'server',
 u'hardwar',
 u'detail',
 u'pretti',
 u'strong',
 u'convict',
 u'involv',
 u'bought',
 u'heavi',
 u'easi']

In [635]:
sample = documents[documents['index'] == 1035].values[0][0]

print "original document: "
words = []
for word in sample.split(" "):
    words.append(word)
print words
print " "
print "tokenized and lemmatized document: "
print tokenization(sample)

original document: 
['I', 'did', 'something', 'similar', 'with', 'AMD', '', 'it', '', 'generally', '', 'has', 'positive', 'sentiment', 'and', 'I', 'knew', 'Google', 's', 'game', 'streaming', 'keynote', 'was', 'coming', 'up', 'and', 'would', 'talk', 'server', 'hardware', 'details', '', 'since', 'I', 'had', 'a', 'pretty', 'strong', 'conviction', 'that', 'AMD', 'would', 'be', 'involved', 'I', 'bought', 'in', 'heavy', 'and', 'made', 'an', 'easy', '10', '', '', '', '']
 
tokenized and lemmatized document: 
[u'similar', u'general', u'posit', u'sentiment', u'knew', u'googl', u'game', u'stream', u'keynot', u'come', u'talk', u'server', u'hardwar', u'detail', u'pretti', u'strong', u'convict', u'involv', u'bought', u'heavi', u'easi']


In [636]:
documents['comments'].head(10)

0    I think this selloff will bleed into next week...
1    What if the sell off was    them    selling of...
2    Nope  Saw someone buy puts at the ask on futur...
3    This interface is super confusing  Just tell u...
4        Yolo Microsoft put  Made another 13k on RH   
5                                    nice  bro        
6    What do you think of APPL calls before their M...
7    Puts on everything  Down Friday means gap down...
8                             Yolo puts  Seriously    
9    I will close all calls monday morning and put ...
Name: comments, dtype: object

In [637]:
processed_documents = documents['comments'].map(tokenization)
processed_documents[:10]

0    [think, selloff, bleed, week, look, selloff, l...
1     [sell, sell, news, hour, later, mueller, report]
2                  [nope, put, futur, think, contract]
3    [interfac, super, confus, tell, posit, total, ...
4                                    [yolo, microsoft]
5                                               [nice]
6    [think, appl, call, march, announc, servic, play]
7    [put, friday, mean, monday, close, monday, ope...
8                                 [yolo, put, serious]
9                     [close, call, monday, morn, put]
Name: comments, dtype: object

## Bag of Words dataset on documents 

Using the BoW dataset to return number of times a word appears in the training set.

In [638]:
dictionary = gensim.corpora.dictionary.Dictionary(processed_documents)

count = 0
for key, value in sorted(dictionary.iteritems()):
    print key,value
    count+=1
    if count>700:
        break

0 bleed
1 come
2 dumb
3 fear
4 gonna
5 know
6 like
7 long
8 look
9 loom
10 money
11 recess
12 right
13 selloff
14 think
15 time
16 week
17 hour
18 later
19 mueller
20 news
21 report
22 sell
23 contract
24 futur
25 nope
26 put
27 confus
28 gain
29 interfac
30 posit
31 super
32 tell
33 total
34 microsoft
35 yolo
36 nice
37 announc
38 appl
39 call
40 march
41 play
42 servic
43 close
44 friday
45 low
46 lower
47 mean
48 monday
49 open
50 test
51 serious
52 morn
53 day
54 flip
55 indic
56 peopl
57 reliabl
58 yeah
59 gap
60 technic
61 comfort
62 complac
63 fomo
64 histor
65 pig
66 real
67 slaughter
68 sold
69 thought
70 today
71 chart
72 csco
73 cvna
74 daili
75 disgust
76 grmn
77 liter
78 market
79 overextend
80 predict
81 shop
82 short
83 way
84 broke
85 follow
86 wrong
87 set
88 broseph
89 rememb
90 commiss
91 lose
92 correct
93 etrad
94 need
95 option
96 switch
97 ameritrad
98 better
99 cost
100 cover
101 stock
102 trade
103 usual
104 forward
105 good
106 manag
107 msft
108 pretti
109 se

In [639]:
# filter out tokens that appear in 
# a. less than 21 documents
# b. above 0.5 documents
# c. keep the first 600 most frequent tokens
dictionary.filter_extremes(no_below=21, no_above=0.5, keep_n=600)

In [640]:
count=0
for key, value in sorted(dictionary.iteritems(), reverse=True):
    print key,value
    count+=1
    if count>10:
        break

599 brand
598 roth
597 beat
596 estat
595 premium
594 typic
593 april
592 boomer
591 phone
590 advic
589 contribut


In [641]:
data.title.value_counts()

Daily advice thread All questions about your personal situation should be asked here                                                                                                                                                                                                               614
After Two Of The Greatest Bull Markets In U S History Why Are Boomers So Broke                                                                                                                                                                                                                     551
Random discussion thread Anything goes                                                                                                                                                                                                                                                             301
Zillow 's House Flipping Averages An Abysmal 0 6 Per Flip                                                          

In [690]:
data[data.title.str.contains("Mcdonald s buys Dynamic Yield for 300 million")].head()

,commentsUrl,comments,date,date_str,linkFlair,score,subreddit,title
26437,https://old.reddit.com/r/investing/comments/b6...,Article a href https www reuters com art...,"datetime.datetime(2019, 3, 28, 9, 59, 20, 217...",'2019-03-28',Discussion,429,investing,Mcdonald s buys Dynamic Yield for 300 million
26438,https://old.reddit.com/r/investing/comments/b6...,It s not mentioned in the article but they wil...,"datetime.datetime(2019, 3, 28, 9, 59, 20, 217...",'2019-03-28',Discussion,429,investing,Mcdonald s buys Dynamic Yield for 300 million
26439,https://old.reddit.com/r/investing/comments/b6...,I think the big metric to watch to see how wel...,"datetime.datetime(2019, 3, 28, 9, 59, 20, 217...",'2019-03-28',Discussion,429,investing,Mcdonald s buys Dynamic Yield for 300 million
26440,https://old.reddit.com/r/investing/comments/b6...,Anecdotal but people feel more comfortable ad...,"datetime.datetime(2019, 3, 28, 9, 59, 20, 217...",'2019-03-28',Discussion,429,investing,Mcdonald s buys Dynamic Yield for 300 million
26441,https://old.reddit.com/r/investing/comments/b6...,So if its cold and dreary out their drive thro...,"datetime.datetime(2019, 3, 28, 9, 59, 20, 217...",'2019-03-28',Discussion,429,investing,Mcdonald s buys Dynamic Yield for 300 million


In [647]:
documents.comments[1843]

'You ve lost money in stocks since Oct  2016    '

In [643]:
len(bow_corpus)

10093

Create a dictionary for each document indicating how many times a word appears, save this to 'bow_corpus'

In [648]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_documents]
bow_18445 = bow_corpus[1843]

In [649]:
# checking how often 
for i in range(len(bow_18445)):
    print "Word {}, {}, appears {} times.".format(bow_18445[i][0],
                                                  dictionary[bow_18445[i][0]],
                                                             bow_18445[i][1])

Word 8, money, appears 1 times.
Word 70, stock, appears 1 times.
Word 140, lost, appears 1 times.


create a tf-idf model (term frequency - inverted document frequency) model and save it to tf-idf, apply a transformation to the corpus and call it corpus_tfidf

In [650]:

tfidf = gensim.models.TfidfModel(bow_corpus) # fitting the model
corpus_tfidf = tfidf[bow_corpus] # apply the model to all documents

In [651]:
# previewing tf-idf scores for the first document
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.2460471416858584),
 (1, 0.3743501837965858),
 (2, 0.3809654829088901),
 (3, 0.31328859695201416),
 (4, 0.20895227905708416),
 (5, 0.15448071769055557),
 (6, 0.21467153700736602),
 (7, 0.19464745234114117),
 (8, 0.190681302932228),
 (9, 0.270261835801707),
 (10, 0.22571363377346496),
 (11, 0.18355526665480057),
 (12, 0.3844783278201842),
 (13, 0.2503554854921157)]


## Training the LDA Model with Bag of Words dataset 

In [652]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)


In [653]:
for idx, topic in lda_model.print_topics(-1):
    print "Topic: {},\n Words:{}".format(idx,topic)

Topic: 0,
 Words:0.027*"money" + 0.024*"compani" + 0.024*"like" + 0.019*"great" + 0.018*"good" + 0.017*"peopl" + 0.014*"hope" + 0.012*"start" + 0.012*"hous" + 0.012*"point"
Topic: 1,
 Words:0.071*"like" + 0.030*"work" + 0.016*"peopl" + 0.016*"busi" + 0.015*"hard" + 0.015*"yeah" + 0.015*"idea" + 0.014*"think" + 0.014*"know" + 0.014*"mean"
Topic: 2,
 Words:0.054*"year" + 0.034*"think" + 0.030*"compani" + 0.028*"market" + 0.022*"go" + 0.017*"month" + 0.016*"stock" + 0.015*"happen" + 0.011*"buy" + 0.010*"need"
Topic: 3,
 Words:0.043*"thank" + 0.038*"time" + 0.033*"put" + 0.032*"call" + 0.026*"post" + 0.019*"hold" + 0.018*"bought" + 0.016*"think" + 0.016*"contract" + 0.016*"nice"
Topic: 4,
 Words:0.036*"price" + 0.023*"make" + 0.022*"right" + 0.018*"valu" + 0.017*"loan" + 0.013*"invest" + 0.013*"option" + 0.012*"sens" + 0.012*"rate" + 0.012*"check"
Topic: 5,
 Words:0.039*"long" + 0.037*"term" + 0.027*"short" + 0.024*"strong" + 0.023*"recess" + 0.022*"yield" + 0.019*"bond" + 0.019*"year" + 0

### The topics using Bag of Words technique are a bit hard to determine, in this case we'll attempt to train LDA using TF-IDF 

In [654]:
lda_model_tfidf = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=4)


In [655]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print "Topic: {},\n Words:{}".format(idx,topic)

Topic: 0,
 Words:0.025*"fuck" + 0.018*"like" + 0.018*"market" + 0.016*"think" + 0.016*"year" + 0.016*"dividend" + 0.014*"loan" + 0.013*"return" + 0.012*"work" + 0.012*"better"
Topic: 1,
 Words:0.032*"market" + 0.032*"stock" + 0.025*"long" + 0.024*"sell" + 0.019*"week" + 0.017*"hous" + 0.016*"today" + 0.013*"time" + 0.012*"go" + 0.012*"know"
Topic: 2,
 Words:0.051*"compani" + 0.025*"know" + 0.024*"stock" + 0.018*"peopl" + 0.015*"think" + 0.015*"point" + 0.013*"thought" + 0.013*"mean" + 0.012*"like" + 0.012*"make"
Topic: 3,
 Words:0.029*"money" + 0.028*"short" + 0.027*"http" + 0.027*"like" + 0.024*"href" + 0.018*"cash" + 0.017*"nofollow" + 0.017*"market" + 0.017*"fund" + 0.016*"hold"
Topic: 4,
 Words:0.029*"year" + 0.028*"term" + 0.023*"long" + 0.019*"price" + 0.019*"call" + 0.019*"option" + 0.018*"go" + 0.017*"right" + 0.016*"thank" + 0.016*"like"
Topic: 5,
 Words:0.027*"think" + 0.026*"like" + 0.022*"peopl" + 0.015*"go" + 0.014*"card" + 0.013*"price" + 0.013*"pretti" + 0.012*"market" +

In [656]:
data['title'][384]

'Glad I sold these yesterday before close Would ve been worthless if I still held '

In [657]:
documents['comments'][357]

'So  AMZN was at 1712 at Monday open and then acquired more than 100 points before Friday    '

# Performance evaluation of LDA models on processed documents.

In [658]:
for index, score in sorted(lda_model[bow_corpus[357]], key=lambda tupl: -1*tupl[1]):
    print "Score: {} \nTopic: {}\n".format(score, lda_model.print_topic(index,10))

Score: 0.819945335388 
Topic: 0.043*"thank" + 0.038*"time" + 0.033*"put" + 0.032*"call" + 0.026*"post" + 0.019*"hold" + 0.018*"bought" + 0.016*"think" + 0.016*"contract" + 0.016*"nice"

Score: 0.0200093649328 
Topic: 0.027*"money" + 0.024*"compani" + 0.024*"like" + 0.019*"great" + 0.018*"good" + 0.017*"peopl" + 0.014*"hope" + 0.012*"start" + 0.012*"hous" + 0.012*"point"

Score: 0.0200089477003 
Topic: 0.048*"appl" + 0.036*"peopl" + 0.034*"good" + 0.023*"thing" + 0.023*"like" + 0.016*"think" + 0.014*"servic" + 0.013*"probabl" + 0.012*"want" + 0.012*"edit"

Score: 0.0200087055564 
Topic: 0.046*"money" + 0.038*"fuck" + 0.025*"account" + 0.025*"stock" + 0.022*"shit" + 0.017*"card" + 0.017*"news" + 0.016*"get" + 0.016*"want" + 0.016*"market"

Score: 0.0200071353465 
Topic: 0.039*"long" + 0.037*"term" + 0.027*"short" + 0.024*"strong" + 0.023*"recess" + 0.022*"yield" + 0.019*"bond" + 0.019*"year" + 0.017*"loss" + 0.017*"rate"

Score: 0.0200047716498 
Topic: 0.054*"year" + 0.034*"think" + 0.03

## Performance evaluation of LDA model using TF-IDF 

In [659]:
documents['comments'][356]

'He bought them at the beginning of the week   '

In [660]:
data['title'][359]

'Glad I sold these yesterday before close Would ve been worthless if I still held '

In [661]:
for index, score in sorted(lda_model_tfidf[bow_corpus[356]], key=lambda tupl: -1*tupl[1]):
    print "Score: {} \nTopic: {}\n".format(score, lda_model_tfidf.print_topic(index,10))

Score: 0.774913132191 
Topic: 0.032*"market" + 0.032*"stock" + 0.025*"long" + 0.024*"sell" + 0.019*"week" + 0.017*"hous" + 0.016*"today" + 0.013*"time" + 0.012*"go" + 0.012*"know"

Score: 0.0250198002905 
Topic: 0.052*"time" + 0.027*"strong" + 0.022*"year" + 0.013*"like" + 0.013*"close" + 0.013*"post" + 0.012*"thank" + 0.012*"month" + 0.011*"sell" + 0.011*"mayb"

Score: 0.0250167082995 
Topic: 0.029*"money" + 0.028*"short" + 0.027*"http" + 0.027*"like" + 0.024*"href" + 0.018*"cash" + 0.017*"nofollow" + 0.017*"market" + 0.017*"fund" + 0.016*"hold"

Score: 0.0250149760395 
Topic: 0.027*"think" + 0.026*"like" + 0.022*"peopl" + 0.015*"go" + 0.014*"card" + 0.013*"price" + 0.013*"pretti" + 0.012*"market" + 0.011*"look" + 0.011*"rate"

Score: 0.0250102430582 
Topic: 0.029*"year" + 0.028*"term" + 0.023*"long" + 0.019*"price" + 0.019*"call" + 0.019*"option" + 0.018*"go" + 0.017*"right" + 0.016*"thank" + 0.016*"like"

Score: 0.0250065661967 
Topic: 0.036*"like" + 0.032*"money" + 0.015*"month" + 

# Testing LDA TF-IDF on unprocessed data

In [684]:
unseen_document = "Amazon"
bow_vector = dictionary.doc2bow(tokenization(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.549956798553	 Topic: 0.027*"think" + 0.026*"like" + 0.022*"peopl" + 0.015*"go" + 0.014*"card"
Score: 0.0500101149082	 Topic: 0.036*"like" + 0.032*"money" + 0.015*"month" + 0.014*"need" + 0.013*"market"
Score: 0.0500078015029	 Topic: 0.051*"compani" + 0.025*"know" + 0.024*"stock" + 0.018*"peopl" + 0.015*"think"
Score: 0.0500065423548	 Topic: 0.052*"time" + 0.027*"strong" + 0.022*"year" + 0.013*"like" + 0.013*"close"
Score: 0.0500062070787	 Topic: 0.045*"invest" + 0.030*"appl" + 0.024*"thing" + 0.016*"need" + 0.015*"like"
Score: 0.0500052757561	 Topic: 0.032*"market" + 0.032*"stock" + 0.025*"long" + 0.024*"sell" + 0.019*"week"
Score: 0.0500024147332	 Topic: 0.054*"trade" + 0.041*"good" + 0.024*"bond" + 0.022*"market" + 0.021*"stock"
Score: 0.0500018335879	 Topic: 0.029*"money" + 0.028*"short" + 0.027*"http" + 0.027*"like" + 0.024*"href"
Score: 0.0500016361475	 Topic: 0.029*"year" + 0.028*"term" + 0.023*"long" + 0.019*"price" + 0.019*"call"
Score: 0.0500013828278	 Topic: 0.025*"f

In [686]:
unseen_document = "Microsoft"
bow_vector = dictionary.doc2bow(tokenization(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.10000000149	 Topic: 0.025*"fuck" + 0.018*"like" + 0.018*"market" + 0.016*"think" + 0.016*"year"
Score: 0.10000000149	 Topic: 0.032*"market" + 0.032*"stock" + 0.025*"long" + 0.024*"sell" + 0.019*"week"
Score: 0.10000000149	 Topic: 0.051*"compani" + 0.025*"know" + 0.024*"stock" + 0.018*"peopl" + 0.015*"think"
Score: 0.10000000149	 Topic: 0.029*"money" + 0.028*"short" + 0.027*"http" + 0.027*"like" + 0.024*"href"
Score: 0.10000000149	 Topic: 0.029*"year" + 0.028*"term" + 0.023*"long" + 0.019*"price" + 0.019*"call"
Score: 0.10000000149	 Topic: 0.027*"think" + 0.026*"like" + 0.022*"peopl" + 0.015*"go" + 0.014*"card"
Score: 0.10000000149	 Topic: 0.052*"time" + 0.027*"strong" + 0.022*"year" + 0.013*"like" + 0.013*"close"
Score: 0.10000000149	 Topic: 0.054*"trade" + 0.041*"good" + 0.024*"bond" + 0.022*"market" + 0.021*"stock"
Score: 0.10000000149	 Topic: 0.036*"like" + 0.032*"money" + 0.015*"month" + 0.014*"need" + 0.013*"market"
Score: 0.10000000149	 Topic: 0.045*"invest" + 0.030*"app

In [682]:
unseen_document = "Gold"
bow_vector = dictionary.doc2bow(tokenization(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.549921274185	 Topic: 0.027*"think" + 0.026*"like" + 0.022*"peopl" + 0.015*"go" + 0.014*"card"
Score: 0.0500220023096	 Topic: 0.052*"time" + 0.027*"strong" + 0.022*"year" + 0.013*"like" + 0.013*"close"
Score: 0.0500118546188	 Topic: 0.054*"trade" + 0.041*"good" + 0.024*"bond" + 0.022*"market" + 0.021*"stock"
Score: 0.050010945648	 Topic: 0.045*"invest" + 0.030*"appl" + 0.024*"thing" + 0.016*"need" + 0.015*"like"
Score: 0.0500089451671	 Topic: 0.032*"market" + 0.032*"stock" + 0.025*"long" + 0.024*"sell" + 0.019*"week"
Score: 0.0500074774027	 Topic: 0.029*"year" + 0.028*"term" + 0.023*"long" + 0.019*"price" + 0.019*"call"
Score: 0.0500049926341	 Topic: 0.051*"compani" + 0.025*"know" + 0.024*"stock" + 0.018*"peopl" + 0.015*"think"
Score: 0.0500045754015	 Topic: 0.025*"fuck" + 0.018*"like" + 0.018*"market" + 0.016*"think" + 0.016*"year"
Score: 0.0500042065978	 Topic: 0.029*"money" + 0.028*"short" + 0.027*"http" + 0.027*"like" + 0.024*"href"
Score: 0.0500036925077	 Topic: 0.036*"lik

In [689]:
unseen_document = "Apple"
bow_vector = dictionary.doc2bow(tokenization(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.549983978271	 Topic: 0.045*"invest" + 0.030*"appl" + 0.024*"thing" + 0.016*"need" + 0.015*"like"
Score: 0.0500056371093	 Topic: 0.027*"think" + 0.026*"like" + 0.022*"peopl" + 0.015*"go" + 0.014*"card"
Score: 0.0500030145049	 Topic: 0.051*"compani" + 0.025*"know" + 0.024*"stock" + 0.018*"peopl" + 0.015*"think"
Score: 0.0500019192696	 Topic: 0.032*"market" + 0.032*"stock" + 0.025*"long" + 0.024*"sell" + 0.019*"week"
Score: 0.0500012598932	 Topic: 0.036*"like" + 0.032*"money" + 0.015*"month" + 0.014*"need" + 0.013*"market"
Score: 0.0500011965632	 Topic: 0.029*"year" + 0.028*"term" + 0.023*"long" + 0.019*"price" + 0.019*"call"
Score: 0.0500009134412	 Topic: 0.025*"fuck" + 0.018*"like" + 0.018*"market" + 0.016*"think" + 0.016*"year"
Score: 0.050000756979	 Topic: 0.052*"time" + 0.027*"strong" + 0.022*"year" + 0.013*"like" + 0.013*"close"
Score: 0.0500007234514	 Topic: 0.029*"money" + 0.028*"short" + 0.027*"http" + 0.027*"like" + 0.024*"href"
Score: 0.050000615418	 Topic: 0.054*"trad

Model has a strong preference for APPL, Gold, Amazon but little to say about Microsoft due to the lack of data. 